In [1]:
%load_ext autoreload
%autoreload 2

# BTM用に､10000分割したテキストをre-clusteringするための分配処理

In [2]:
#loaderの設定などの練習
import json
from loaders.CommonCrawlDataset import CommonCrawlDataset

In [3]:
#10000分割されたjsonlファイルのパスを取得

import glob
import random

jsonl_path_list=glob.glob("../data/dedup_categorized/*/*.jsonl")
random.shuffle(jsonl_path_list)
cid_to_path_list={}
for jsonl_path in jsonl_path_list:
    cid=jsonl_path.split("/")[-2]
    if cid not in cid_to_path_list:
        cid_to_path_list[cid]=[]
    cid_to_path_list[cid].append(jsonl_path)

In [4]:
from tqdm import tqdm
#代表的な*件を取得
max_texts=10
cid_to_texts={}
for cid, path_list in tqdm(cid_to_path_list.items()):
    texts=[]
    for path in path_list:
        cnt=0
        with open(path) as f:
            for i, line in enumerate(f):
                t=json.loads(line)["text"]
                if len(t)<10:
                    continue
                texts.append(t)
                cnt+=1
                if cnt>max_texts:
                    break
    cid_to_texts[cid]=texts

100%|██████████| 9813/9813 [02:38<00:00, 61.89it/s]  


In [5]:
import sys
import numpy as np
sys.path.append("../01web_codes")
from gensim.models.fasttext import load_facebook_model
from tqdm import tqdm
from src.classify.Text2Vec import Text2Vec

#t2v = Text2Vec(load_facebook_model('../data/model/cc.ja.300.bin'))

In [6]:
from gensim.models import KeyedVectors
t2v = Text2Vec(model=KeyedVectors.load_word2vec_format(
    '../data/model/entity_vector/entity_vector.model.bin', binary=True),
    dim=200,
)

In [7]:
#各カテゴリのテキストをベクトル化
cid_to_vecs={}
check_length=100
for cid, texts in tqdm(cid_to_texts.items()):
    vecs = [t2v.text2vec(i[:check_length]) for i in texts]
    vecs=np.array(vecs)
    vecs=np.mean(vecs, axis=0)
    cid_to_vecs[cid]=vecs



100%|██████████| 9813/9813 [08:11<00:00, 19.98it/s] 


In [14]:
vec_array=list(cid_to_vecs.values())
vec_array=[i for i in vec_array if i.shape==(200,)]
vec_array=np.array(vec_array).astype(np.double)
vec_array.shape

(9813, 200)

In [15]:
n_clusters=5
from sklearn.cluster import MiniBatchKMeans,KMeans
# k-meansクラスタリング
print("clustering...")
kmeans = KMeans(n_clusters=n_clusters, random_state=1).fit(vec_array)

clustering...


In [16]:
cid_to_label={}
for cid, vec in cid_to_vecs.items():
    try:
        label=kmeans.predict([vec])[0]
        cid_to_label[cid]=label
    except Exception as e:
        print(cid,e)

In [17]:
label_to_path_list={}
for cid, label in cid_to_label.items():
    if str(label) not in label_to_path_list:
        label_to_path_list[str(label)]=[]
    path=f"../data/dedup_categorized/{cid}/*.jsonl"
    label_to_path_list[str(label)].append(path)


In [18]:
import json
with open("../data/clustered_path.json", "w") as f:
    json.dump(label_to_path_list, f, indent=4)

In [19]:
#モデル保存
import pickle
with open("../data/second_kmeans_model.pkl", "wb") as f:
    pickle.dump(kmeans, f)

In [22]:
#表示する
n_clusters=5
from loaders.CommonCrawlDataset import CommonCrawlDataset
for i in range(n_clusters):
    print("--------\ncluster: ",i)
    cc1_loader=CommonCrawlDataset(label_to_path_list[str(i)])
    cc1_iter=iter(cc1_loader)
    for i in range(15):
        print(next(cc1_iter))

--------
cluster:  0
{'text': 'もしも180日が経過してもボランティアを行わなかった場合は、この4ドルはカード発行団体に振り込まれる。そういうふうになっていく。'}
{'text': '生成した鍵ファイルをメニューの下にある公開鍵の保存ボタン、秘密鍵の保存ボタンを押して保存先を指定することで鍵を保存できます。ほんとはProgram Files(x86)/teratermに置きたいんですが、teratermから直接置くことができないので、一旦デスクトップやマイドキュメントに保存します。\n保存した鍵ファイル2つをc:\\Program Files(x86)\\teratermに移動します。\n選択肢が出てきますがSSH接続であるSCPを選びましょう。\n鍵がちゃんとあるか確認してみます。'}
{'text': '先に安い車をレベル5にして、パーツ割引発生させるとかやった方がいいかもしれないよ。'}
{'text': '割り当てインデックス内で、ファイルはアルファベット順に割り当てられるため、NTLDRファイルが2番目の割り当てインデックスに押し出されることがあります。その結果、NTLDR is missingエラーメッセージが表示されることがあります。一般にファイルがルートフォルダに書き込まれることはありません。'}
{'text': 'でも、NHKの取材班がカメラを回していましたので、もしかしたら秋田では放送されるかもしれません。'}
{'text': 'キャッシングカード発行の申込書の記載内容に、書き漏れや書き間違いがあれば、ほぼすべてのケースで再提出か訂正印を押さなければいけないので、あなたに返却されてしまうことになります。本人が自分で記入すべき箇所は、じっくりと確認するなど慎重に取り扱ってください。'}
{'text': '独立に発達したネットワーク(NW)を融合させるには、動的安定性と整合性を保ちながらNW及びNW間にどのような信号伝達の制御が行われているかを知る必要があります。この問題解決に向け、独立に進化してきた遺伝子代謝NWもつ2種の生物を用いて、共生関係の進化と、その過程で起こるNWの再編成について研究します。'}
{'text': 'のような、ビット列を示す文字が表示されるという具合です。これで受信側のプログラムはでき